In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import glob
import PIL
import gc
from PIL import Image
from sklearn .model_selection import StratifiedKFold, GroupKFold

import warnings
warnings.filterwarnings('ignore')

In [2]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _list_float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _list_int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


In [3]:
RESIZE_WIDTH = 512
RESIZE_HEIGHT = 512

In [4]:
df_train = pd.read_parquet(f'../input/shope-text-vectorizer-for-all-train/train.parquet', engine='pyarrow')

samples = df_train
n_samples = len(samples)
print(f'{n_samples} samples')
with tf.io.TFRecordWriter(f'/kaggle/working/train.tfrec') as writer:
    for row in samples.itertuples():
        image = f'../input/shopee-product-matching/train_images/{row.image}'
        img = open(image, 'rb')
        img_read = img.read()
        image_decoded = tf.image.decode_jpeg(img_read, channels=3)
        resized_img = tf.image.resize_with_pad(image_decoded,target_width=RESIZE_WIDTH,target_height=RESIZE_HEIGHT,method=tf.image.ResizeMethod.BILINEAR)
        resized_img = tf.cast(resized_img,tf.uint8)
        resized_img = tf.io.encode_jpeg(resized_img)

        feature = {
            'filename': _bytes_feature(tf.compat.as_bytes(row.image)),
            'label': _int64_feature(row.label_group),
            'title_vec': _list_float_feature(row.title_vec),
            'phash_vec': _list_float_feature(row.phash_vec),
            'image': _bytes_feature(resized_img),
            'height' : _int64_feature(RESIZE_HEIGHT),
            'width' : _int64_feature(RESIZE_WIDTH)
        }

        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())

34250 samples


In [5]:
!ls -lah

total 2.3G
drwxr-xr-x 2 root root 4.0K May  9 20:40 .
drwxr-xr-x 6 root root 4.0K May  9 20:39 ..
---------- 1 root root 7.1K May  9 20:48 __notebook__.ipynb
-rw-r--r-- 1 root root 2.3G May  9 20:48 train.tfrec
